# CNN Image Classification Laboration


Images used in this laboration are from CIFAR 10 (https://en.wikipedia.org/wiki/CIFAR-10). The CIFAR-10 dataset contains 60,000 32x32 color images in 10 different classes. The 10 different classes represent airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. There are 6,000 images of each class. Your task is to make a classifier, using a convolutional neural network, that can correctly classify each image into the correct class.

You need to answer all questions in this notebook. 

## Part 1: What is a convolution

To understand a bit more about convolutions, we will first test the convolution function in scipy using a number of classical filters. 

Convolve the image with Gaussian filter, a Sobel X filter, and a Sobel Y filter, using the function 'convolve2d' in 'signal' from scipy.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html

In a CNN, many filters are applied in each layer, and the filter coefficients are learned through back propagation.

In [1]:
from scipy import signal
import numpy as np

# Get a test image
from scipy import misc
image = misc.ascent()

# Define a help function for creating a Gaussian filter
def matlab_style_gauss2D(shape=(3,3),sigma=0.5):
    """
    2D gaussian mask - should give the same result as MATLAB's
    fspecial('gaussian',[shape],[sigma])
    """
    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h

# Create Gaussian filter with certain size and standard deviation
gaussFilter = matlab_style_gauss2D((15,15),4)

# Define filter kernels for SobelX and Sobely
sobelX = np.array([[ 1, 0,  -1],
                    [2, 0, -2],
                    [1, 0, -1]]) 

sobelY = np.array([[ 1, 2,  1],
                    [0, 0, 0],
                    [-1, -2, -1]]) 

In [2]:
# Perform convolution using the function 'convolve2d' for the different filters
filterResponseGauss  = signal.convolve2d(image, gaussFilter, mode='valid')
filterResponseSobelX = signal.convolve2d(image, sobelX, mode='valid')
filterResponseSobelY = signal.convolve2d(image, sobelY, mode='valid')

In [ ]:
# Create Gaussian filter with certain size and standard deviation
gaussFilter = matlab_style_gauss2D((15,15),4)

# Define filter kernels for SobelX and Sobely
sobelX = np.array([[ 1, 0,  -1],
                    [2, 0, -2],
                    [1, 0, -1]]) 

sobelY = np.array([[ 1, 2,  1],
                    [0, 0, 0],
                    [-1, -2, -1]]) 


In [ ]:
# Show filter responses
import matplotlib.pyplot as plt
fig, (ax_orig, ax_filt1, ax_filt2, ax_filt3) = plt.subplots(1, 4, figsize=(20, 6))
ax_orig.imshow(image, cmap='gray')
ax_orig.set_title('Original')
ax_orig.set_axis_off()
ax_filt1.imshow(np.absolute(filterResponseGauss), cmap='gray')
ax_filt1.set_title('Filter response')
ax_filt1.set_axis_off()
ax_filt2.imshow(np.absolute(filterResponseSobelX), cmap='gray')
ax_filt2.set_title('Filter response')
ax_filt2.set_axis_off()
ax_filt3.imshow(np.absolute(filterResponseSobelY), cmap='gray')
ax_filt3.set_title('Filter response')
ax_filt3.set_axis_off()

##  Part 2: Understanding convolutions

Question 1: What do the 3 different filters (Gaussian, SobelX, SobelY) do to the original image?

Question 2: What is the size of the original image? How many channels does it have? How many channels does a color image normally have?

Question 3: What is the size of the different filters?

Question 4: What is the size of the filter response if mode 'same' is used for the convolution ?

Question 5: What is the size of the filter response if mode 'valid' is used for the convolution? How does the size of the valid filter response depend on the size of the filter? 

Question 6: Why are 'valid' convolutions a problem for CNNs with many layers?


In [ ]:
# Your code for checking sizes of image and filter responses
print("Image size : {}".format(image.shape))
print("\nFilter response size in 'valid' mode :" +
      "\n\tGaussian filter response : {}".format(filterResponseGauss.shape) +
      "\n\tSobelX / SobelY filter response : {}".format(filterResponseSobelX.shape))

filterResponseGauss2  = signal.convolve2d(image, gaussFilter, mode='same')
filterResponseSobelX2 = signal.convolve2d(image, sobelX, mode='same')
filterResponseSobelY2 = signal.convolve2d(image, sobelY, mode='same')

print("\nFilter response size in 'same' mode :" +
      "\n\tGaussian filter response : {}".format(filterResponseGauss2.shape) +
      "\n\tSobelX / SobelY filter response : {}".format(filterResponseSobelX2.shape))


### Answers Q1-Q6

`Q1:` Gaussian filter blurs the image and reduces contrast. The sobelX and sobelY filters detect edges in the vertical and horizontal directions respectively.

`Q2:` Original image is of size 512 x 512. Since all the images in the dataset are color images, the original image has 3 channels. Any color image normally has 3 channels, 1 for each of the RGB colors.

`Q3:` Gaussian filter is of size 15 x 15, sobelX and sobelY filters are both of size 3 x 3.

`Q4:` The gaussian, sobelX and sobelY filter responses all have the same size as that of the original image when mode 'same' is used for convolution. 

`Q5:` The gaussian filter of size 15 x 15 gives a response of size 498 x 498 and the sobelX/Y filters of size 3 x 3 give response of size 510 x 510 when mode 'valid' is used for convolution on an image of size 512 x 512. The valid convolution gives output only when the filter completely overlaps with the image and we will get a full overlap only when the center of the filter is atleast $(f-1)/2$ (for odd filter size $f$) away from the top/bottom and right/left edges of the image. So the size of the response will be $i-(f-1) = i-f+1$ where $i$ is the image size and $f$ is the odd filter size.

`Q6:` From above we know that a valid convolution reduces the size of the input image. In a CNN with many layers, the output from one valid convolution will be fed as an input for the next convolution and the size of the output will go on diminishing. With many layers and hence repeated convolutions, we might run out of pixels in the original image. 

##  Part 3: Get a graphics card

Let's make sure that our script can see the graphics card that will be used. The graphics cards will perform all the time consuming convolutions in every training iteration.

In [ ]:
import os
import warnings

# Ignore FutureWarning from numpy
warnings.simplefilter(action='ignore', category=FutureWarning)

import keras.backend as K
import tensorflow as tf

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

# Allow growth of GPU memory, otherwise it will always look like all the memory is being used
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True) 

##  Part 4: How fast is the graphics card?

Lets investigate how much faster a convolution is with the graphics card

Question 7: Why are the filters of size 7 x 7 x 3, and not 7 x 7 ? 

Question 8: What operation is performed by the 'Conv2D' layer? Is it a standard 2D convolution, as performed by the function signal.convolve2d we just tested?

Question 9: How much faster is the graphics card, compared to the CPU, for convolving a batch of 100 images?

Question 10: How much faster is the graphics card, compared to the CPU, for convolving a batch of 2 images? Explain the difference compared to 100 images.

### Answers Q7-Q10

`Q7:` The filters are of size 7 x 7 x 3 because the input image has three color (RGB) channels and the filter operates seperately on each of the input channels. The filter is a collection of kernels, one for every input channel to the layer. The kernel outputs for each channel are summed to produce one overall output channel

`Q8:` The Conv2D layer performs a spatial convolution over 2D images. It is the same operation as performed by the function signal.convolve2d i.e. it is a linear combination of series of 2D image data with a few coefficients or weights. 

`Q9:` CPU=0.993915017000063, GPU=0.19508830099994157, GPU speedup over CPU = 5x. The GPU speedup over CPU is quite good in this case. 

`Q10:` CPU=0.06825254199975461, GPU=0.05537720200027252, GPU speedup over CPU = 1x. The GPU does not give any significant performance improvement over CPU when the batch size=2. This is because when the batch size is too small, even though we are utilizing the parallelization capabilities of the GPU, the no of iterations in each epoch increases which increases the training time. 

In [ ]:
# Run this cell to compare processing time of CPU and GPU

import timeit

n_images_in_batch = 1

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

# Perform convolutions using the CPU
def cpu():
  with tf.device('/cpu:0'):
    random_images = tf.random.normal((n_images_in_batch, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_images)
    return tf.math.reduce_sum(net_cpu)

# Perform convolutions using the GPU (graphics card)
def gpu():
  with tf.device('/device:GPU:0'):
    random_images = tf.random.normal((n_images_in_batch, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_images)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the convolution several times and measure the time
print('Time (s) to convolve 32 filters of size 7 x 7 x 3 over 100 random images of size 100 x 100 x 3'
      ' (batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

##  Part 5:  Load data
Time to make a 2D CNN. Load the images and labels from keras.datasets, this cell is already finished.

In [ ]:
from keras.datasets import cifar10
import numpy as np

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Download CIFAR train and test data
(Xtrain, Ytrain), (Xtest, Ytest) = cifar10.load_data()

print("Training images have size {} and labels have size {} ".format(Xtrain.shape, Ytrain.shape))
print("Test images have size {} and labels have size {} \n ".format(Xtest.shape, Ytest.shape))

# Reduce the number of images for training and testing to 10000 and 2000 respectively, 
# to reduce processing time for this laboration
Xtrain = Xtrain[0:10000]
Ytrain = Ytrain[0:10000]

Xtest = Xtest[0:2000]
Ytest = Ytest[0:2000]

Ytestint = Ytest

print("Reduced training images have size %s and labels have size %s " % (Xtrain.shape, Ytrain.shape))
print("Reduced test images have size %s and labels have size %s \n" % (Xtest.shape, Ytest.shape))

# Check that we have some training examples from each class
for i in range(10):
    print("Number of training examples for class {} is {}" .format(i,np.sum(Ytrain == i)))

In [ ]:
help(cifar10.load_data)

##  Part 6: Plotting

Lets look at some of the training examples, this cell is already finished. You will see different examples every time you run the cell.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,4))
for i in range(18):
    idx = np.random.randint(7500)
    label = Ytrain[idx,0]
    
    plt.subplot(3,6,i+1)
    plt.tight_layout()
    plt.imshow(Xtrain[idx])
    plt.title("Class: {} ({})".format(label, classes[label]))
    plt.axis('off')
plt.show()

##   Part 7: Split data into training, validation and testing
Split your training data into training (Xtrain, Ytrain) and validation (Xval, Yval), so that we have training, validation and test datasets (as in the previous laboration). We use a function in scikit learn. Use 25% of the data for validation.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
from sklearn.model_selection import train_test_split

# Split training data into training and validation datasets
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain, Ytrain, test_size=0.25)

# Print the size of training data, validation data and test data
print("Training images have size %s and labels have size %s " % (Xtrain.shape, Ytrain.shape))
print("Validation images have size %s and labels have size %s" % (Xval.shape, Yval.shape))
print("Test images have size %s and labels have size %s \n" % (Xtest.shape, Ytest.shape))

##  Part 8: Preprocessing of images

Lets perform some preprocessing. The images are stored as uint8, i.e. 8 bit unsigned integers, but need to be converted to 32 bit floats. We also make sure that the range is -1 to 1, instead of 0 - 255. This cell is already finished.

In [ ]:
# Convert datatype for Xtrain, Xval, Xtest, to float32
Xtrain = Xtrain.astype('float32')
Xval = Xval.astype('float32')
Xtest = Xtest.astype('float32')

# Change range of pixel values to [-1,1]
Xtrain = Xtrain / 127.5 - 1
Xval = Xval / 127.5 - 1
Xtest = Xtest / 127.5 - 1

##  Part 9: Preprocessing of labels

The labels (Y) need to be converted from e.g. '4' to "hot encoded", i.e. to a vector of type [0, 0, 0, 1, 0, 0, 0, 0, 0, 0] . We use a function in Keras, see https://keras.io/utils/#to_categorical

In [ ]:
from keras.utils import to_categorical

# Print shapes before converting the labels
print("Training image labels have size {}".format(Ytrain.shape))
print("Validation image labels have size {}".format(Yval.shape))
print("Test image labels have size {}".format(Ytest.shape))

# Your code for converting Ytrain, Yval, Ytest to categorical
Ytrain = to_categorical(y=Ytrain, num_classes=10)
Yval   = to_categorical(y=Yval)
Ytest  = to_categorical(y=Ytest)

# Print shapes after converting the labels
print("\nTraining image labels have size {}".format(Ytrain.shape))
print("Validation image labels have size {}".format(Yval.shape))
print("Test image labels have size {}".format(Ytest.shape))

##  Part 10: 2D CNN
Finish this code to create the image classifier, using a 2D CNN. Each convolutional layer will contain 2D convolution, batch normalization and max pooling. After the convolutional layers comes a flatten layer and a number of intermediate dense layers. The convolutional layers should take the number of filters as an argument, use a kernel size of 3 x 3, 'same' padding, and relu activation functions. The number of filters will double with each convolutional layer. The max pooling layers should have a pool size of 2 x 2. The intermediate dense layers before the final dense layer should take the number of nodes as an argument, use relu activation functions, and be followed by batch normalization. The final dense layer should have 10 nodes (= the number of classes in this laboration) and 'softmax' activation. Here we start with the Adam optimizer.

Relevant functions are

`model.add()`, adds a layer to the network

`Dense()`, a dense network layer

`Conv2D()`, performs 2D convolutions with a number of filters with a certain size (e.g. 3 x 3). 

`BatchNormalization()`, perform batch normalization

`MaxPooling2D()`, saves the max for a given pool size, results in down sampling

`Flatten()`, flatten a multi-channel tensor into a long vector

`model.compile()`, compile the model, add " metrics=['accuracy'] " to print the classification accuracy during the training

See https://keras.io/layers/core/ for information on how the `Dense()` and `Flatten()` functions work

See https://keras.io/layers/convolutional/ for information on how `Conv2D()` works

See https://keras.io/layers/pooling/ for information on how `MaxPooling2D()` works

Import a relevant cost function for multi-class classification from keras.losses (https://keras.io/losses/)

See https://keras.io/models/model/ for how to compile, train and evaluate the model

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy as CC

# Set seed from random number generator, for better comparisons
from numpy.random import seed
seed(123)

def build_CNN(input_shape, n_conv_layers=2, n_filters=16, n_dense_layers=0, n_nodes=50, use_dropout=False, learning_rate=0.01):

    # Setup a sequential model
    model = Sequential()

    # Add first convolutional layer to the model, requires input shape
    model.add(Conv2D(filters=n_filters, kernel_size=(3,3), padding='same', activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2,2)))
        
    # Add remaining convolutional layers to the model, the number of filters should increase a factor 2 for each layer
    for i in range(n_conv_layers-1):
        n_filters = n_filters*2
        model.add(Conv2D(filters=n_filters, kernel_size=(3,3), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
                         
    # Add flatten layer
    model.add(Flatten())
    
    # Add intermediate dense layers
    for i in range(n_dense_layers):
        model.add(Dense(units=n_nodes, activation='relu'))
        model.add(BatchNormalization())
        
        if use_dropout==True:
            model.add(Dropout(rate=0.5))
        
    # Add final dense layer
    model.add(Dense(units=10, activation='softmax'))
    
    # Compile model
    model.compile(loss=CC, optimizer=Adam(learning_rate=learning_rate), metrics=["categorical_accuracy"])
    
    return model

In [ ]:
# Lets define a help function for plotting the training results
import matplotlib.pyplot as plt
def plot_results(history):
    
    loss = history.history['loss']
    acc = history.history['categorical_accuracy']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_categorical_accuracy']
    
    plt.figure(figsize=(10,4))
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(loss)
    plt.plot(val_loss)
    plt.legend(['Training','Validation'])

    plt.figure(figsize=(10,4))
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(acc)
    plt.plot(val_acc)
    plt.legend(['Training','Validation'])

    plt.show()

## Part 11: Train 2D CNN

Time to train the 2D CNN, start with 2 convolutional layers, no intermediate dense layers, learning rate = 0.01. The first convolutional layer should have 16 filters (which means that the second convolutional layer will have 32 filters).

Relevant functions

`build_CNN`, the function we defined in Part 10, call it with the parameters you want to use

`model.fit()`, train the model with some training data

`model.evaluate()`, apply the trained model to some test data

## 2 convolutional layers, no intermediate dense layers

In [ ]:
# Setup some training parameters
batch_size = 100
epochs = 20
input_shape = (32, 32, 3)

# Build model
model1 = build_CNN(input_shape=input_shape, n_conv_layers=2, n_filters=16, n_dense_layers=0, learning_rate=0.01)

# Train the model using training data and validation data
history1 = model1.fit(x=Xtrain, y=Ytrain, batch_size=batch_size, epochs=epochs, validation_data=(Xval, Yval))

In [ ]:
# Evaluate the trained model on test set, not used in training or validation
score = model1.evaluate(x=Xtest, y=Ytest, batch_size=batch_size)
print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
# Plot the history from the training run
plot_results(history1)

## Part 12: Improving performance
Write down the test accuracy, are you satisfied with the classifier performance (random chance is 10%) ? 

Question 11: How big is the difference between training and test accuracy?

Question 12: How busy is the GPU for a batch size of 100? How much GPU memory is used? Hint: run 'watch nvidia-smi' on the cloud computer during training. 

Question 13: For the DNN laboration we used a batch size of 10,000, why do we need to use a smaller batch size in this laboration?

### Answers Q11-Q13

`Q11:` We achieved a training accuracy of 0.94 and test accuracy of 0.53. The test accuracy is lower than the training by 0.41 i.e. almost by half. It can be said that the model has an acceptable classification accuracy considering 10% as random chance. 

`Q12:` 

`Q13:` 

## 2 convolutional layers, 1 intermediate dense layer (50 nodes)

In [ ]:
# Setup some training parameters
batch_size = 100
epochs = 20
input_shape = (32, 32, 3)

# Build model
model2 = build_CNN(input_shape=input_shape, n_conv_layers=2, n_filters=16, n_dense_layers=1, n_nodes=50, learning_rate=0.01)

# Train the model  using training data and validation data
history2 = model2.fit(x=Xtrain, y=Ytrain, batch_size=batch_size, epochs=epochs, validation_data=(Xval, Yval))

In [ ]:
# Evaluate the trained model on test set, not used in training or validation
score = model2.evaluate(x=Xtest, y=Ytest, batch_size=batch_size)
print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
# Plot the history from the training run
plot_results(history2)

## 4 convolutional layers, 1 intermediate dense layer (50 nodes)

In [ ]:
# Setup some training parameters
batch_size = 100
epochs = 20
input_shape = (32, 32, 3)

# Build model
model3 = build_CNN(input_shape=input_shape, n_conv_layers=4, n_filters=16, n_dense_layers=1, n_nodes=50, learning_rate=0.01)

# Train the model  using training data and validation data
history3 = model3.fit(x=Xtrain, y=Ytrain, batch_size=batch_size, epochs=epochs, validation_data=(Xval, Yval))

In [ ]:
# Evaluate the trained model on test set, not used in training or validation
score = model3.evaluate(x=Xtest, y=Ytest, batch_size=batch_size)
print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
# Plot the history from the training run
plot_results(history3)

##  Part 13: Plot the CNN architecture

To understand your network better, print the architecture using `model.summary()`

Question 14: How many trainable parameters does your network have? Which part of the network contains most of the parameters?

Question 15: What is the input to and output of a Conv2D layer? What are the dimensions of the input and output? 

Question 16: Is the batch size always the first dimension of each 4D tensor? Check the documentation for Conv2D, https://keras.io/layers/convolutional/

Question 17: If a convolutional layer that contains 128 filters is applied to an input with 32 channels, what is the number of channels in the output?

Question 18: Why is the number of parameters in each Conv2D layer *not* equal to the number of filters times the number of filter coefficients per filter (plus biases)?

Question 19: How does MaxPooling help in reducing the number of parameters to train?

### Answers Q14-Q19

`Q14:` The CNN has a total of 200,500 trainable parameters. The last convolutional layer contains the most number of parameters. The last Conv2D layer has 128 filters of size 3 x 3 and hence 3 x 3 x 64 weights (64 is the number of input channels to the last Conv2D layer) and 1 bias for each filter. Therefore, this layer has ((3 x 3 x 64) + 1) x 128 = 73856 parameters in total.  

The dense layer contains the next most number of parameters as every neuron is connected to every other neuron from the previous layer. The fully connected layer has 50 nodes each connected to all of the 512 nodes from the flattened layer. Hence it has 50 x 512 weights and 50 biases = 25650 total parameters.

`Q15:` The input to a Conv2D layer is an input image with channels and the output is a convolved image with (usually higher) output channels. These convolved image channels are called feature maps. Each output channel (feature map) is the output of one filter applied to the previous layer. If the input dimensions to a conv2D layer with $n_{filter}$ filters are $i_{width}$ x $i_{height}$ x $i_{channels}$ then the output of the layer has dimenions $i_{width}$ x $i_{height}$ x $n_{filter}$.

`Q16:` Yes, the first dimension of a 4D tensor is always the batch_size. It does not depend on the data_format parameter of the Conv2D layer. If the data_format = channels_last, then the dimension of the 4D tensor is (batch_size, rows, cols, channels) and if the data_format = channels_first, then the dimension of the 4D tensor is (batch_size, channels, rows, cols). 

`Q17:` The number of output channels is the same as the number of filters in the convolutional layer. Each of the 128 filters in the Conv2D layer has 32 filter kernels (equal to the number of input channels). So we will get one output channel for each filter which is the sum of all the 32 kernels in that filter convolved with the 32 input channels respectively. 

`Q18:` The number of filter coefficients per filter is $f_{width}$ x $f_{height}$ but each filter has depth (filter kernels) equal to the number of input channels i.e. $f_{kernels}$=$i_{channels}$. The filter output is the sum over all the kernel outputs. Hence the number of parameters in a Conv2D layer = (($f_{width}$ x $f_{height}$ x $f_{kernels}$ + 1) x $n_{filter}$ where $f_{kernels}$=$i_{channels}$. The 1 is for a bias per filter.   

`Q19:` MaxPooling layer with pool size=2 reduces the output dimension by half of the input dimension. For every 2 x 2 neighbourhood in the input image, we take the maximum value as the output. The default stride is equal to the pool size. This reduces the output dimension and hence the parameters associated with the successive layers in the network. MaxPooling itself doesn't have any learnable parameters. 

In [ ]:
# Print network architecture
model3.summary()

##  Part 14: Dropout regularization

Add dropout regularization to each intermediate dense layer, dropout probability 50%.

Question 20: How much did the test accuracy improve with dropout, compared to without dropout?

Question 21: What other types of regularization can be applied? How can you add L2 regularization for the convolutional layers?


### Answers Q20-Q21

`Q20:` The test accuracy of the model increased approximately by 0.1 with dropout regularization. The test accuracy of the CNN model without dropout is 0.558 and with dropout is 0.565 

`Q21:` Bias and kernel regularization, or image augmentation can also be applied to the CNN. 

## 4 convolutional layers, 1 intermediate dense layer (50 nodes), dropout

In [ ]:
# Setup some training parameters
batch_size = 100
epochs = 20
input_shape = (32, 32, 3)

# Build model
model4 = build_CNN(input_shape=input_shape, n_conv_layers=4, n_filters=16, n_dense_layers=1, n_nodes=50, 
                   use_dropout=True, learning_rate=0.01)

# Train the model  using training data and validation data
history4 = model4.fit(x=Xtrain, y=Ytrain, batch_size=batch_size, epochs=epochs, validation_data=(Xval, Yval))

In [ ]:
# Evaluate the trained model on test set, not used in training or validation
score = model4.evaluate(x=Xtest, y=Ytest, batch_size=batch_size)
print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
# Plot the history from the training run
plot_results(history4)

##  Part 15: Tweaking performance

You have now seen the basic building blocks of a 2D CNN. To further improve performance involves changing the number of convolutional layers, the number of filters per layer, the number of intermediate dense layers, the number of nodes in the intermediate dense layers, batch size, learning rate, number of epochs, etc. Spend some time (30 - 90 minutes) testing different settings.

Question 22: How high test accuracy can you obtain? What is your best configuration?

### Answer Q22

`Q22:` We are able to get an accuracy of upto 0.62 on test images with the below configuration:    

convolutional layers: 4   
max pool layers: 4   
intermediate dense layer: 1  
number of nodes in dense layer: 80  


## Your best config

In [ ]:
# Setup some training parameters
batch_size = 100
epochs = 20
input_shape = (32, 32, 3)

# Build model
model5 = build_CNN(input_shape=input_shape, n_conv_layers=4, n_filters=16, n_dense_layers=1, n_nodes=80, 
                   use_dropout=True, learning_rate=0.01)

# Train the model  using training data and validation data
history5 = model5.fit(x=Xtrain, y=Ytrain, batch_size=batch_size, epochs=epochs, validation_data=(Xval, Yval))

In [ ]:
# Evaluate the trained model on test set, not used in training or validation
score = model5.evaluate(x=Xtest, y=Ytest, batch_size=batch_size)
print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
# Plot the history from the training run
plot_results(history5)

##  Part 16: Rotate the test images

How high is the test accuracy if we rotate the test images? In other words, how good is the CNN at generalizing to rotated images?

Rotate each test image 90 degrees, the cells are already finished.

Question 23: What is the test accuracy for rotated test images, compared to test images without rotation? Explain the difference in accuracy.

### Answer Q23

`Q23:` The test accuracy of the model for rotated images is 0.23 which is quite low. It is not very better than classifying images at random chance. The CNN model is trained to look for the learned features in only a certain orientations. This is why it results in a low test accuracy. 

In [ ]:
def myrotate(images):

    images_rot = np.rot90(images, axes=(1,2))
    
    return images_rot

In [ ]:
# Rotate the test images 90 degrees
Xtest_rotated = myrotate(Xtest)

# Look at some rotated images
plt.figure(figsize=(16,4))
for i in range(10):
    idx = np.random.randint(500)
    
    plt.subplot(2,10,i+1)
    plt.imshow(Xtest[idx]/2+0.5)
    plt.title("Original")
    plt.axis('off')
    
    plt.subplot(2,10,i+11)
    plt.imshow(Xtest_rotated[idx]/2+0.5)
    plt.title("Rotated")
    plt.axis('off')
plt.show()


In [ ]:
# Evaluate the trained model on rotated test set
score = model5.evaluate(x=Xtest_rotated, y=Ytest, batch_size=batch_size)
print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

##  Part 17: Augmentation using Keras `ImageDataGenerator`

We can increase the number of training images through data augmentation (we now ignore that CIFAR10 actually has 60 000 training images). Image augmentation is about creating similar images, by performing operations such as rotation, scaling, elastic deformations and flipping of existing images. This will prevent overfitting, especially if all the training images are in a certain orientation.

We will perform the augmentation on the fly, using a built-in function in Keras, called `ImageDataGenerator`

See https://keras.io/preprocessing/image/

In [ ]:
# Get all 60 000 training images again. ImageDataGenerator manages validation data on its own
(Xtrain, Ytrain), _ = cifar10.load_data()

# Reduce number of images to 10,000
Xtrain = Xtrain[0:10000]
Ytrain = Ytrain[0:10000]

# Change data type and rescale range
Xtrain = Xtrain.astype('float32')
Xtrain = Xtrain / 127.5 - 1

# Convert labels to hot encoding
Ytrain = to_categorical(Ytrain, 10)

In [ ]:
# Set up a data generator with on-the-fly data augmentation, 20% validation split
# Use a rotation range of 30 degrees, horizontal and vertical flipping
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=30, horizontal_flip=True, vertical_flip=True, validation_split=0.2)

# Setup a flow for training data, assume that we can fit all images into CPU memory
datagen_train = datagen.flow(x=Xtrain, y=Ytrain, batch_size=len(Xtrain)*0.8, subset="training")

# Setup a flow for validation data, assume that we can fit all images into CPU memory
datagen_valid = datagen.flow(x=Xtrain, y=Ytrain, batch_size=len(Xtrain)*0.2, subset="validation")

## Part 18: What about big data?

Question 24: How would you change the code for the image generator if you cannot fit all training images in CPU memory? What is the disadvantage of doing that change?

### Answer Q24

`Q24:` We can generate the images in batches i.e. reduce the batch_size. 

In [ ]:
# Plot some augmented images
plot_datagen = datagen.flow(Xtrain, Ytrain, batch_size=1)

plt.figure(figsize=(12,4))
for i in range(18):
    (im, label) = plot_datagen.next()
    im = (im[0] + 1) * 127.5
    im = im.astype('int')
    label = np.flatnonzero(label)[0]
    
    plt.subplot(3,6,i+1)
    plt.tight_layout()
    plt.imshow(im)
    plt.title("Class: {} ({})".format(label, classes[label]))
    plt.axis('off')
plt.show()

## Part 19: Train the CNN with images from the generator

See https://keras.io/models/model/ for how to use model.fit_generator instead of model.fit for training

To make the comparison fair to training without augmentation

    steps_per_epoch should be set to: len(Xtrain)*(1 - validation_split)/batch_size

    validation_steps should be set to: len(Xtrain)*validation_split/batch_size

Question 25: How quickly is the training accuracy increasing compared to without augmentation? Explain why there is a difference compared to without augmentation. What parameter is necessary to change to perform more training?

Question 26: What other types of image augmentation can be applied, compared to what we use here?

### Answers Q25-Q26

`Q25:`
    
`Q26:` Images can also be cropped, translated (shifted left, right, up or down) and images can also be mixed.

In [ ]:
# Setup some training parameters
batch_size = 100
epochs = 20
input_shape = (32, 32, 3)

# Build model (your best config)
model6 = build_CNN(input_shape=input_shape, n_conv_layers=4, n_filters=16, n_dense_layers=1, n_nodes=80, 
                   use_dropout=True, learning_rate=0.01)

validation_split=0.2
    
# Train the model using on the fly augmentation
datagen_train = datagen.flow(x=Xtrain, y=Ytrain, batch_size=batch_size, subset="training")
datagen_valid = datagen.flow(x=Xtrain, y=Ytrain, batch_size=batch_size, subset="validation")

history6 = model6.fit_generator(generator=datagen_train, steps_per_epoch=len(Xtrain)*0.8/batch_size, epochs=epochs,
                                validation_data=datagen_valid, validation_steps=len(Xtrain)*0.2/batch_size)

In [ ]:
# Check if there is still a big difference in accuracy for original and rotated test images

# Evaluate the trained model on original test set
score = model6.evaluate(Xtest, Ytest, batch_size = batch_size, verbose=0)
print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

# Evaluate the trained model on rotated test set
score = model6.evaluate(Xtest_rotated, Ytest, batch_size = batch_size, verbose=0)
print('Test loss: %.4f' % score[0])
print('Test accuracy: %.4f' % score[1])

In [ ]:
# Plot the history from the training run
plot_results(history6)

## Part 20: Plot misclassified images

Lets plot some images where the CNN performed badly

In [ ]:
# Find misclassified images
y_pred = model6.predict_classes(Xtest)
y_correct = np.argmax(Ytest,axis=1)

miss = np.flatnonzero(y_correct != y_pred)

In [ ]:
# Plot a few of them
plt.figure(figsize=(15,4))
perm = np.random.permutation(miss)
for i in range(18):
    im = (Xtest[perm[i]] + 1) * 127.5
    im = im.astype('int')
    label_correct = y_correct[perm[i]]
    label_pred = y_pred[perm[i]]
    
    plt.subplot(3,6,i+1)
    plt.tight_layout()
    plt.imshow(im)
    plt.axis('off')
    plt.title("{}, classified as {}".format(classes[label_correct], classes[label_pred]))
plt.show()

##  Part 21: Testing on another size

Question 27: This CNN has been trained on 32 x 32 images, can it be applied to images of another size? If not, why is this the case?

Question 28: Is it possible to design a CNN that can be trained on images of one size, and then applied to an image of any size? How?

### Answers Q27-Q28

`Q27:` No, our CNN trained on 32 x 32 images cannot be applied to images of another size because it is not a fully convolutional network. We have intermediate dense layers. In case of a dense layer, its parameters are predefined. Hence the input dimensions which goes to the dense layer are fixed. 

`Q28` Yes, a fully convolutional network can be trained on images of size A and then applied on images of size B. A fully convolutional network contains only convolutional layers and does not have any dense layers. Instead it has 1 x 1 convolutions that perform the task of fully connected layers. The input to the 1 x 1 convolution layer could be anything as long as the number of filters in 1 x 1 convolution layer (i.e. input to the final dense layer) is equal to the number of classes.

##  Part 22: Pre-trained 2D CNNs

There are many deep 2D CNNs that have been pre-trained using the large ImageNet database. Import a pre-trained ResNet50 network from Keras applications. Show the network using `model.summary()`

Question 29: How many convolutional layers does ResNet50 have? 

Question 30: How many trainable parameters does the ResNet50 network have? 

Question 31: What is the size of the images that ResNet50 expects as input?

Question 32: Using the answer to question 30, explain why the second derivative is seldom used when training deep networks.

Apply the pre-trained CNN to 5 random color images that you download and copy to the cloud machine. Are the predictions correct? How certain is the network of each image class?

These pre-trained networks can be fine tuned to your specific data, and normally only the last layers need to be re-trained, but it will still be too time consuming to do in this laboration.

See https://keras.io/applications/#resnet

Useful functions

`image.load_img` in keras.preprocessing

`image.img_to_array` in keras.preprocessing

`ResNet50` in keras.applications.resnet50

`preprocess_input` in keras.applications.resnet50

`decode_predictions` in keras.applications.resnet50

`expand_dims` in numpy


### Answers Q29-Q32

`Q29:`

`Q30:`

`Q31:`

`Q32:`

In [ ]:
# Your code for using pre-trained ResNet 50 on 5 color images of your choice
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image
import PIL
import numpy as np

model = ResNet50(include_top=True, weights='imagenet')

img_path = 
img = load_img(img_path, target_size=(224, 224))

x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

model.summary()


In [ ]:
help(load_img)